In [11]:
import pandas as pd
import glob
import os
import pickle

### System-level data

In [12]:
wmt18_no_metadata = pickle.load(open('../data/pickles/wmt18-sys_level-all.pkl', 'rb'))
wmt18_no_metadata.count()

system       244337
sid          244337
raw_score    244337
score        244337
N            244337
lp           244337
reference    244337
source       244337
output       228409
dtype: int64

In [13]:
write = open('./data/ad-raw.csv', 'wt')
read = open('./data/newstest2018-humaneval/analysis/ad-good-raw.csv', 'rt')

for line in read:
    if '+' not in line:
        write.write(line.replace(' ', '\t'))
    else:
        line = line.split('\t')
        system = line[6]
        for i in system.split('+'):
            l = '\t'.join(line[0:6]) + '\t' + i + '\t' + '\t'.join(line[7:])
            write.write(l)
            #break
write.close()

In [14]:
df = pd.read_csv('./data/ad-raw.csv', sep='\t')
df.count()

/nas/home/jwei/anaconda3/envs/notebook/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


HITId         236098
WorkerId      354803
Input.src     354803
Input.trg     354803
Input.item    354803
hit           236098
sys_id        354803
rid           236098
type          354803
sid           354803
score         354803
time          354803
dtype: int64

In [15]:
df['lp'] = df.apply(lambda x: '%s-%s' % (x['Input.src'], x['Input.trg']), axis=1)
df.head(1)

,HITId,WorkerId,Input.src,Input.trg,Input.item,hit,sys_id,rid,type,sid,score,time,lp
0,NaN,engdeu5621,en,de,ad,NaN,LMU-unsup.5651,NaN,BAD_REF,1192,1,16.285,en-de


In [16]:
wmt18_metadata = df.merge(wmt18_no_metadata[['system', 'sid', 'lp', 'reference', 'source', 'output']],
         left_on=['lp', 'sys_id', 'sid'],
         right_on=['lp', 'system', 'sid'],
         how='left'
        )
wmt18_metadata.count()

HITId         236098
WorkerId      354803
Input.src     354803
Input.trg     354803
Input.item    354803
hit           236098
sys_id        354803
rid           236098
type          354803
sid           354803
score         354803
time          354803
lp            354803
system        352384
reference     352384
source        352384
output        352384
dtype: int64

In [18]:
# entries with missing 
wmt18_metadata[(wmt18_metadata.sys_id == 'REFERENCE')]
# wmt18_metadata[(wmt18_metadata.sys_id != 'REFERENCE') & (wmt18_metadata.output.isna())]
wmt18_metadata[(wmt18_metadata.output.isna()) & (wmt18_metadata.type == 'BAD_REF')]

,HITId,WorkerId,Input.src,Input.trg,Input.item,hit,sys_id,rid,type,sid,score,time,lp,system,reference,source,output
7665,NaN,engdeu5627,en,de,ad,NaN,online-Z.0,NaN,BAD_REF,453,65,37.590,en-de,NaN,NaN,NaN,NaN
7669,NaN,engdeu5627,en,de,ad,NaN,uedin.5770,NaN,BAD_REF,2223,16,16.773,en-de,NaN,NaN,NaN,NaN
7674,NaN,engdeu5627,en,de,ad,NaN,online-F.0,NaN,BAD_REF,387,23,14.521,en-de,NaN,NaN,NaN,NaN
19381,NaN,engdeu5627,en,de,ad,NaN,UCAM.5585,NaN,BAD_REF,2292,37,60.272,en-de,NaN,NaN,NaN,NaN
40831,NaN,engdeu5627,en,de,ad,NaN,KIT.5486,NaN,BAD_REF,1160,24,30.495,en-de,NaN,NaN,NaN,NaN
75514,NaN,engest5017,en,et,ad,NaN,neurotolge.ee.5399,NaN,BAD_REF,1060,21,287.205,en-et,NaN,NaN,NaN,NaN
81472,NaN,deueng5045,de,en,ad,NaN,online-Y.0,NaN,BAD_REF,1558,29,241.425,de-en,NaN,NaN,NaN,NaN
81492,NaN,deueng5045,de,en,ad,NaN,online-F.0,NaN,BAD_REF,1075,9,7.414,de-en,NaN,NaN,NaN,NaN
87057,NaN,esteng5002,et,en,ad,NaN,HY-NMT-et-en.5463,NaN,BAD_REF,1657,14,109.315,et-en,NaN,NaN,NaN,NaN
93627,NaN,engest5017,en,et,ad,NaN,online-A.0,NaN,BAD_REF,905,7,2.154,en-et,NaN,NaN,NaN,NaN


In [8]:
wmt18_metadata = wmt18_metadata[['lp', 'HITId', 'WorkerId', 'score', 'time', 'system', 'type', 'sid', 'reference', 'source', 'output']]

In [9]:
pickle.dump(wmt18_metadata, open('pickles/wmt18_sys_metadata.pkl', 'wb'))